In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
import pandas as pd
import PIL

In [ ]:
conda install nb_conda_kernels

### Train and predict

In [2]:
'''
Define and train a convolutional neural networ with 3 convolutional layers, 3 max-pooling layers
'''
def train_model(data_dir, epochs, seed): 
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split= 0.2,
      subset= "training",
      seed= seed)
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=0.2,
      subset= "validation",
      seed= seed)
    
    class_names = train_ds.class_names
    num_classes = len(class_names)
    print("Class names: {}".format(class_names))
    
    # cache the data, to have it readily available, and to avoid reloading between epochs
    train_ds = train_ds.cache().shuffle(200).prefetch(buffer_size=200)

    multiclass_model = Sequential([
      # rescale the vector of numbers representing color from [0, 255] to [0, 1]
      layers.experimental.preprocessing.Rescaling(1./255),
      # define the convolutional layer, padding and type of activation function
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(num_classes)
    ])
    
    # compile the model, select the type of optimizer and the target metric
    multiclass_model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    # fit the model
    history = multiclass_model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=epochs
    )
    
    return multiclass_model

In [3]:
def predict(path, model, class_names, silent = True):
    img = tf.keras.preprocessing.image.load_img(path,  target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    img_class = class_names[np.argmax(score)]
    prob = 100* np.max(score)
    if not silent:
        print("This image most likely belongs to {} with a {:02f} percent confidence.".format(img_class, prob))
    return score.numpy()

### Binary Classifiers

In [4]:
classes = ['floor', 'frame', 'furniture', 'grass', 'sea', 'sky', 'street', 'window']
test = ['guitar', 'salad', 'book', 'cartoon']
data_path = r"..\data\binary_models"
test_path = r"..\data\test"

class_names = {
    "floor": ["floor", "other"],
    "frame": ["frame", "other"],
    "furniture": ["furniture", "other"],
    "grass": ["grass", "other"],
    "sea": ["other", "sea"],
    "sky": ["other", "sky"], 
    "street": [ "other", "street"],
    "window": [ "other", "window"]
}

models = { }

for c in classes:
    model = train_model(os.path.join(data_path, str(c)+"_binary"), 10, 456)
    models[c] = model
    print("##################################")
    print("Predictions are for model {0}. The first one is the positive class, and the second one the negative class".format(c))
    predict(os.path.join(test_path, str(c)+".jpg"), model, class_names[c])
    predict(os.path.join(test_path, test[np.random.randint(0, 3)]+".jpg"), model, class_names[c])

AttributeError: module 'tensorflow.python.keras.api._v1.keras.preprocessing' has no attribute 'image_dataset_from_directory'

### Snorkel

In [2]:
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import LabelModel

ModuleNotFoundError: No module named 'snorkel.labeling'

In [16]:
ABSTAIN = -1
INDOOR = 0
OUTDOOR = 1

In [17]:
indoor = ['floor', 'frame', 'furniture','window']
outdoor = ['grass', 'sea', 'sky', 'street']

In [18]:
def has_element(image_path, class_name):
    if class_name not in classes:
        raise Exception('Unsupported class')
    
    model = models[class_name]
    score = predict(image_path, model, class_names[class_name])
    predicted_class = class_names[class_name][np.argmax(score)]
    confidence = np.max(score)
    if predicted_class == class_name and confidence > 0.5:
        ret = INDOOR if class_name in indoor else OUTDOOR
        return ret
    else:
        return ABSTAIN

@labeling_function()
def has_grass(x):    
    return has_element(x["Path"], "grass")

@labeling_function()
def has_floor(x):
    return has_element(x["Path"], "floor")

@labeling_function()
def has_furniture(x):
    return has_element(x["Path"], "furniture")

@labeling_function()
def has_frame(x):
    return has_element(x["Path"], "frame")

@labeling_function()
def has_window(x):
    return has_element(x["Path"], "window")

@labeling_function()
def has_sea(x):
    return has_element(x["Path"], "sea")

@labeling_function()
def has_sky(x):
    return has_element(x["Path"], "sky")

@labeling_function()
def has_street(x):
    return has_element(x["Path"], "street")

NameError: name 'labeling_function' is not defined

In [10]:
### Testing our labeling function
for c in classes:
    prediction = has_element(os.path.join(test_path, str(c)+".jpg"), c)
    print("Class {0}. Prediction {1}".format(c, prediction))

Class floor. Prediction 0
Class frame. Prediction 0
Class furniture. Prediction 0
Class grass. Prediction 1
Class sea. Prediction 1
Class sky. Prediction 1
Class street. Prediction 1
Class window. Prediction -1


In [139]:
data_dir = pathlib.Path(r"\images\final")
image_list= list(data_dir.glob('*/*.jpg'))
random.shuffle(image_list)
len(image_list)

2044

In [140]:
df = pd.DataFrame(image_list, columns = ['Path'])
df.head(3)

,Path
0,C:\Users\sefilipi\Documents\books\snorkel\imag...
1,C:\Users\sefilipi\Documents\books\snorkel\imag...
2,C:\Users\sefilipi\Documents\books\snorkel\imag...


In [142]:
df["label"] = df.apply(lambda x: INDOOR if "indoor" in str(x["Path"]) else OUTDOOR, axis=1)
df.head(3)

,Path,label
0,C:\Users\sefilipi\Documents\books\snorkel\imag...,1
1,C:\Users\sefilipi\Documents\books\snorkel\imag...,1
2,C:\Users\sefilipi\Documents\books\snorkel\imag...,1


In [3]:
df = pd.read_csv(r"\images\ccv_v2.csv")

In [4]:
df.shape

(2044, 4)

In [12]:
df_train = df[0:1635]
df_valid = df[1636:2043]

In [13]:
lfs = [
    has_grass,
    has_floor,
    has_furniture,
    has_frame,
    has_window,
    has_sea,
    has_sky,
    has_street,
]

applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
L_valid = applier.apply(df_valid)

C:\Users\sefilipi\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|████████████████████████████████████████████████████████████████████████████████| 407/407 [04:38<00:00,  1.46it/s]


In [184]:
for c in classes:
    score = predict(r"\images\final\85.jpg", models[c], class_names[c])
    print("{0} {1}".format(c, score))

floor [0.03958143 0.9604186 ]
frame [2.2152396e-07 9.9999976e-01]
furniture [0.06251261 0.93748736]
grass [0.4711598 0.5288402]
sea [0.9793729  0.02062705]
sky [0.9886156  0.01138434]
street [0.00842682 0.99157315]
window [0.9967955  0.00320453]


In [185]:
for c in classes:
    prediction = has_element(r"\images\outdoor\85.jpg", c)
    print("Class {0}. Prediction {1}".format(c, prediction))

Class floor. Prediction 0
Class frame. Prediction 0
Class furniture. Prediction 0
Class grass. Prediction 0
Class sea. Prediction 0
Class sky. Prediction 0
Class street. Prediction 1
Class window. Prediction 0


In [14]:
Y_valid = df_valid["label"].values
LFAnalysis(L_valid, lfs).lf_summary(Y_valid)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
has_grass,0,[1],0.157248,0.152334,0.051597,57,0,0.890625
has_floor,1,[0],0.434889,0.429975,0.157248,0,43,0.000000
has_furniture,2,[0],0.690418,0.683047,0.373464,0,108,0.000000
has_frame,3,[0],0.058968,0.058968,0.017199,0,4,0.000000
has_window,4,[0],0.547912,0.542998,0.253071,0,63,0.000000
has_sea,5,[1],0.469287,0.461916,0.262899,133,0,0.696335
has_sky,6,[1],0.130221,0.130221,0.068796,45,0,0.849057
has_street,7,[1],0.496314,0.476658,0.272727,129,0,0.638614


In [190]:
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, seed=456, lr=0.01, log_freq=10, n_epochs=100)
preds_train_label = label_model.predict(L=L_train)

In [198]:
# accuracy, coverage, precision, recall, f1, f1_micro, f1_macro, fbeta, matthews_corrcoef, roc_auc. 
f1_micro = label_model.score(L_valid, Y_valid, metrics=["f1_micro"])
accuracy = label_model.score(L_valid, Y_valid, metrics=["accuracy"])
recall = label_model.score(L_valid, Y_valid, metrics=["recall"])
precision = label_model.score(L_valid, Y_valid, metrics=["precision"])

print("{} {} {} {}".format(f1_micro, accuracy, recall, precision))

{'f1_micro': 0.7738693467336684} {'accuracy': 0.7738693467336684} {'recall': 0.7738693467336684} {'precision': 1.0}


### Cognitive services labeling 

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

In [201]:
subscription_key = "<KEY>"
endpoint = "<ENDPOINT>"

In [203]:
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
remote_image_url = "https://<STORAGE_ACCOUNT>.blob.core.windows.net/images/15.jpg"

In [233]:
# Call the Cognitive Services API
def describe_image(x):
    tags = computervision_client.tag_image(x).tags
    return [tag.name for tag in tags]

In [236]:
describe_image( "https://<STORAGE_ACCOUNT>.blob.core.windows.net/images/15.jpg")

['outdoor',
 'tree',
 'sky',
 'statue',
 'ground',
 'sculpture',
 'art',
 'building',
 'plant',
 'fountain',
 'playground',
 'stone']

#### Add another column to the dataframe, containing the relative URLS, and use that to extract the tags

In [19]:
def extract_path(path):
    index = path.rfind("indoor")
    if(index == -1):
        index = path.rfind("outdoor")
    return path[index:]

In [214]:
### Populating the DataFrame with the urls of the images
df["rel_url"] = df.apply(lambda x: extract_path(str(x["Path"])), axis=1)
df.head(3)

,Path,label,rel_url
0,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\2150.jpg
1,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\85.jpg
2,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\378.jpg


In [253]:
base_storage_url = "https://<STORAGE_ACCOUNT>.blob.core.windows.net/images/"
tags = [None] * df.size

# inquire for 20 images a minute, than pause, 
# because that is the protocol with the free version of the Computer Vision API

for i, row in df.iterrows():    
        tags[i] = describe_image(base_storage_url+df["rel_url"][i])
        print("counter: {0}. Tags {1}".format(i, tags[i]))        

counter: 0. Tags ['tree', 'outdoor', 'ground', 'bicycle', 'bicycle wheel', 'land vehicle', 'wheel', 'vehicle', 'house', 'stone']
counter: 1. Tags ['grass', 'outdoor', 'sky', 'building', 'window', 'porch', 'house', 'door', 'tree', 'shed', 'siding', 'property', 'cottage', 'home', 'garden buildings', 'plant', 'log cabin', 'real estate', 'shack', 'farmhouse', 'grassy', 'old', 'structure', 'lush', 'roof']
counter: 2. Tags ['outdoor', 'sky', 'ground', 'horse', 'mountain', 'group', 'person', 'people', 'mammal', 'dirt', 'sandy', 'riding']
counter: 3. Tags ['indoor', 'sink', 'cabinetry', 'home appliance', 'countertop', 'drawer', 'home', 'design', 'interior', 'floor', 'cupboard', 'house', 'chest of drawers', 'refrigerator', 'shelf', 'furniture', 'kitchen', 'microwave oven', 'white', 'major appliance', 'oven', 'gas stove', 'room', 'bedroom', 'dollhouse', 'vase', 'tap', 'bathroom', 'cabinet', 'kitchen stove', 'plumbing fixture', 'filing cabinet', 'modern', 'kitchen appliance']
counter: 4. Tags ['t

In [251]:
df["tags"] = tags
df.head(21)

,Path,label,rel_url,tags
0,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\2150.jpg,None
1,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\85.jpg,None
2,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\378.jpg,None
3,C:\Users\sefilipi\Documents\books\snorkel\imag...,-1,indoor\763.jpg,None
4,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\261.jpg,None
5,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\2710.jpg,None
6,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\2199.jpg,None
7,C:\Users\sefilipi\Documents\books\snorkel\imag...,-1,indoor\829.jpg,None
8,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\2731.jpg,None
9,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\1850.jpg,None


In [10]:
df = pd.read_csv(r".\images\ccv_v2.csv")

### Snorkel with a new labeling function

In [11]:
@labeling_function()
def is_outdoor(x):
    if "outdoor" in x["tags"]:
        return OUTDOOR
    
    if "indoor" in x["tags"]:
        return INDOOR
    
    return ABSTAIN

In [12]:
df_train = df[0:1635]
df_valid = df[1636:2043]
lfs = [
    has_grass,
    has_floor,
    has_furniture,
    has_frame,
    has_window,
    has_sea,
    has_sky,
    has_street,
    is_outdoor
]

applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
L_valid = applier.apply(df_valid)

C:\Users\sefilipi\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|████████████████████████████████████████████████████████████████████████████████| 407/407 [03:06<00:00,  2.18it/s]


In [13]:
LFAnalysis(L_valid, lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
has_grass,0,[1],0.555283,0.552826,0.304668
has_floor,1,[0],0.592138,0.592138,0.378378
has_furniture,2,[0],0.651106,0.651106,0.422604
has_frame,3,[0],0.179361,0.179361,0.105651
has_window,4,[0],0.297297,0.297297,0.174447
has_sea,5,[1],0.471744,0.471744,0.245700
has_sky,6,[1],0.294840,0.294840,0.108108
has_street,7,[1],0.312039,0.309582,0.105651
is_outdoor,8,"[0, 1]",0.818182,0.815725,0.388206


In [16]:
Y_valid = df_valid["label"].values
LFAnalysis(L_valid, lfs).lf_summary(Y_valid)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
has_grass,0,[1],0.555283,0.552826,0.304668,151,0,0.668142
has_floor,1,[0],0.592138,0.592138,0.378378,0,76,0.000000
has_furniture,2,[0],0.651106,0.651106,0.422604,0,88,0.000000
has_frame,3,[0],0.179361,0.179361,0.105651,0,12,0.000000
has_window,4,[0],0.297297,0.297297,0.174447,0,29,0.000000
has_sea,5,[1],0.471744,0.471744,0.245700,130,0,0.677083
has_sky,6,[1],0.294840,0.294840,0.108108,102,0,0.850000
has_street,7,[1],0.312039,0.309582,0.105651,95,0,0.748031
is_outdoor,8,"[0, 1]",0.818182,0.815725,0.388206,171,6,0.513514


In [17]:
label_model = LabelModel()
label_model.fit(L_train, seed=456, lr=0.01, log_freq=10, n_epochs=100)
preds_train_label = label_model.predict(L=L_train)
# accuracy, coverage, precision, recall, f1, f1_micro, f1_macro, fbeta, matthews_corrcoef, roc_auc. 
f1_micro = label_model.score(L_valid, Y_valid, metrics=["f1_micro"])
accuracy = label_model.score(L_valid, Y_valid, metrics=["accuracy"])
recall = label_model.score(L_valid, Y_valid, metrics=["recall"])
precision = label_model.score(L_valid, Y_valid, metrics=["precision"])

print("{} {} {} {}".format(f1_micro, accuracy, recall, precision))

{'f1_micro': 0.7537688442211056} {'accuracy': 0.7537688442211056} {'recall': 0.7537688442211056} {'precision': 1.0}


In [33]:
preds_valid_label = label_model.predict(L=L_valid)

In [62]:
snorkel_labels = np.concatenate((preds_train_label,[0], preds_valid_label, [0]))

In [63]:
df["snorkel_labels"] = snorkel_labels

In [64]:
df.to_csv("images_data.csv")

### Later

In [29]:
df = pd.read_csv("images_data.csv")
df.head(3)

,Unnamed: 0,Path,label,rel_url,tags,snorkel_labels
0,0,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\2150.jpg,"['tree', 'outdoor', 'ground', 'bicycle', 'bicy...",0
1,1,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\85.jpg,"['grass', 'outdoor', 'sky', 'building', 'windo...",0
2,2,C:\Users\sefilipi\Documents\books\snorkel\imag...,1,outdoor\378.jpg,"['outdoor', 'sky', 'ground', 'horse', 'mountai...",1


In [35]:
df_train = df[0:1635]
df_valid = df[1636:2043]
lfs = [
    has_grass,
    has_floor,
    has_furniture,
    has_frame,
    has_window,
    has_sea,
    has_sky,
    has_street,
    is_outdoor
]

applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
L_valid = applier.apply(df_valid)
#Y_valid = df_valid["label"].values
LFAnalysis(L_valid, lfs).lf_summary()

C:\Users\sefilipi\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel

  8%|██████▎                                                                        | 130/1635 [01:24<17:06,  1.47it/s]


 16%|████████████▋                                                                  | 262/1635 [02:54<20:52,  1.10it/s]


 24%|███████████████████                                                            | 394/1635 [04:44<13:44,  1.51it/s]


 32%|█████████████████████████▍                                                     | 526/1635 [06:12<11:48,  1.57it/s]


 40%|███████████████████████████████▊                                               | 658/1635 [07:45<10:22,  1.57it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████| 1635/1635 [18:34<00:00,  1.47it/s]

 19%|███████████████▋                                                                 | 79/407 [00:51<03:20,  1.64it/s]


 52%|█████████████████████████████████████████▍                                      | 211/407 [02:25<01:59,  1.64it/s]


 84%|███████████████████████████████████████████████████████████████████▍            | 343/407 [03:52<00:44,  1.45it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 407/407 [04:34<00:00,  1.48it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
has_grass,0,"[0, 1]",1.000000,1.000000,0.710074
has_floor,1,[0],0.390663,0.390663,0.375921
has_furniture,2,[0],0.481572,0.481572,0.457002
has_frame,3,[0],0.906634,0.906634,0.660934
has_window,4,[0],0.442260,0.442260,0.422604
has_sea,5,"[0, 1]",1.000000,1.000000,0.710074
has_sky,6,"[0, 1]",1.000000,1.000000,0.710074
has_street,7,"[0, 1]",1.000000,1.000000,0.710074
is_outdoor,8,"[0, 1]",0.646192,0.646192,0.577396
